In [ ]:
pip install ax-platform

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
import math

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable 
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.core.simple_experiment import SimpleExperiment

In [ ]:
#mount to google drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/2021/paper/garch_data/

In [ ]:
# reading the data
start_time = time.time()
VIX = pd.read_excel('VIX_data.xlsx', converters= {'Date': pd.to_datetime})
garch = pd.read_excel('Total_stock_garch_vol_data_v3.xlsx', converters= {'Date': pd.to_datetime})
egarch = pd.read_excel('Total_stock_egarch_vol_data_v3.xlsx', converters= {'Date': pd.to_datetime})
print("---{}s seconds(read data)---".format(time.time()-start_time))

In [ ]:
def date2index(df):
  df.index = df['Date']
  return df.drop(columns='Date')

def reshaping(nmp):
  (x,y)=nmp.shape
  nmp = nmp.reshape(x,1,y)
  return nmp


def make_train_test(df, VIX, train_size):
  mm = MinMaxScaler()
  ss = StandardScaler()

  df_X_ss = ss.fit_transform(df)
  df_y_mm = mm.fit_transform(VIX)

  X_train = df_X_ss[:train_size, :]
  X_test = df_X_ss[train_size:, :]
  
  y_train = df_y_mm[:train_size, :]
  y_test = df_y_mm[train_size:, :]

  X_train = reshaping(X_train)
  X_test = reshaping(X_test)

  print("Training Shape", X_train.shape, y_train.shape)
  print("Testing Shape", X_test.shape, y_test.shape) 

  return (X_train, y_train, X_test, y_test)


def make_train_test_tensor(df, VIX, train_size):
  mm = MinMaxScaler()
  ss = StandardScaler()

  df_X_ss = ss.fit_transform(df)
  df_y_mm = mm.fit_transform(VIX)

  X_train = df_X_ss[:train_size, :]
  X_test = df_X_ss[train_size:, :]
  
  y_train = df_y_mm[:train_size, :]
  y_test = df_y_mm[train_size:, :]

  X_train_tensors = Variable(torch.Tensor(X_train))
  X_test_tensors = Variable(torch.Tensor(X_test))

  y_train_tensors = Variable(torch.Tensor(y_train))
  y_test_tensors = Variable(torch.Tensor(y_test))

  X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
  X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 

  print("Training tensor Shape", X_train_tensors_final.shape, y_train_tensors.shape)
  print("Testing tensor Shape", X_test_tensors_final.shape, y_test_tensors.shape) 

  return (X_train_tensors_final,X_test_tensors_final,y_train_tensors,y_test_tensors)
  

def dataloader2tensor(dataloader):
  i = 0
  for inputs, labels in train_loader:
    i+=1
    if i == 1:
      x = inputs
      y = labels
    else:
      x = torch.cat((x,inputs),0)
      y = torch.cat((y,labels),0)
  return x, y


def visualize(df, VIX, model, train_size):
  mm = MinMaxScaler()
  ss = StandardScaler()

  df_X_ss = ss.fit_transform(df)
  df_y_mm = mm.fit_transform(VIX)

  #converting to Tensors
  df_X_ss = Variable(torch.Tensor(df_X_ss)) 
  df_y_mm = Variable(torch.Tensor(df_y_mm))

  #reshaping the dataset
  df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], 1, df_X_ss.shape[1]))
  train_predict = model(df_X_ss.to(device))#forward pass
  data_predict = train_predict.data.detach().cpu().numpy() #numpy conversion
  dataY_plot = df_y_mm.data.numpy()

  data_predict = mm.inverse_transform(data_predict) #reverse transformation
  dataY_plot = mm.inverse_transform(dataY_plot)
  plt.figure(figsize=(10,6)) #plotting
  plt.axvline(x=train_size, c='r', linestyle='--') #size of the training set

  plt.plot(dataY_plot, label='Actual Data') #actual plot
  plt.plot(data_predict, label='Predicted Data') #predicted plot
  plt.title('Time-Series Prediction')
  plt.legend()
  plt.show() 
  return dataY_plot, data_predict


def errors(GT, DP, train_size):
  GT = GT[train_size:]
  DP = DP[train_size:]
  #MSE
  MSE = ((DP-GT)**2).sum()/len(DP)
  print("MSE : {}\n".format(MSE))
  #MAE
  MAE = (np.abs(DP-GT)).sum()/len(DP)
  print("MAE : {}\n".format(MAE))
  #MAPE
  MAPE = (np.abs(1-DP/GT)).sum()/len(DP)
  print("MAPE : {}\n".format(MAPE))
  #RMSE
  RMSE = math.sqrt(((DP-GT)**2).sum()/len(DP))
  print("RMSE : {}".format(RMSE))
  return (MSE, MAE, MAPE, RMSE)


def int2act_func(num):
  if num == 0:
    activation_function = nn.Tanh()
  elif num == 1:
    activation_function = nn.Sigmoid()
  elif num == 2:
    activation_function = nn.ReLU()
  elif num == 3:
    activation_function = nn.Softsign()
  elif num == 4:
    activation_function = nn.ELU()
  return activation_function

def num2act(num):
  if num == 0:
    act = "Tanh"
  elif num == 1:
    act = "Sigmoid"
  elif num == 2:
    act = "ReLU"
  elif num == 3:
    act = "Softsign"
  elif num == 4:
    act = "ELU"
  return act


def int2optim_func(num, net, lr):
  if num == 0:
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
  elif num == 1:
    optimizer = torch.optim.AdamW(net.parameters(), lr=lr)
  elif num == 2:
    optimizer = torch.optim.Adagrad(net.parameters(), lr=lr)
  elif num == 3:
    optimizer = torch.optim.ASGD(net.parameters(), lr=lr)
  return optimizer

def num2optim(num):
  if num == 0:
    optim = "Adam"
  elif num == 1:
    optim = "AdamW"
  elif num == 2:
    optim = "Adagrad"
  elif num == 3:
    optim = "ASGD"
  return optim

class CustomDataset(Dataset): 
  def __init__(self, input, label):
    self.x_data = input
    self.y_data = label

  def __len__(self): 
    return len(self.x_data)

  def __getitem__(self, idx): 
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

def evaluate(
    net: nn.Module, data_loader: DataLoader, dtype: torch.dtype, device: torch.device
) -> float:
    net.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            total += 1
            inputs = inputs.to(dtype=dtype, device=device)
            labels = labels.to(device=device)
            outputs = net(inputs)
            correct += (outputs - labels)**2

    print("MSE eval : {}\ntest total : {}".format((correct/total).item(), total))

    return (correct/total).item()

In [ ]:
def net_train(net, train_loader, parameters, dtype, device):
  start_time = time.time()

  net.to(dtype=dtype, device=device)

  lr = parameters.get("lr")
  num_epochs = parameters.get("num_epochs") * 100
  optim_num = parameters.get("optim_num")

  print("num_epoch : {}\n".format(num_epochs))
  print("lr : {}\n".format(lr))
  print("optim_func : {}\n".format(num2optim(optim_num)))

  # Define loss, optimizer and dataloader
  criterion = torch.nn.MSELoss()
  optimizer = int2optim_func(optim_num, net, lr)
  (x, y) = dataloader2tensor(train_loader)

  print("trian x : {} , y : {}".format(x.shape,y.shape))

  # Train Network
  for epoch in range(num_epochs):
    outputs = net.forward(x.to(device))
    optimizer.zero_grad()

    # obtain the loss function
    loss = criterion(outputs, y.to(device))

    loss.backward()
    optimizer.step()

  print("total epoch time : {}".format(time.time()-start_time))

  return net

In [ ]:
class LSTM_Time_Series(nn.Module):
  def __init__(self, parameters):
    super(LSTM_Time_Series, self).__init__()
    self.num_classes = parameters.get("num_classes")
    self.num_layers = parameters.get("num_layers")
    self.input_size = parameters.get("input_size")
    self.hidden_size = parameters.get("hidden_size")

    num_classes = parameters.get("num_classes")
    num_layers = parameters.get("num_layers")
    input_size = parameters.get("input_size")
    hidden_size = parameters.get("hidden_size")
    act_num = parameters.get("act_num")

    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                      num_layers = num_layers, batch_first=True)
    self.fc_1 =  nn.Linear(hidden_size, 128)
    self.fc = nn.Linear(128, num_classes)
    self.act = int2act_func(act_num)

    print("hidden_size : {}".format(hidden_size))
    print("activation_function : {}\n".format(num2act(act_num)))

  def forward(self,x):
    h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
    c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state   

    # Propagate input through LSTM
    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next

    out = self.act(hn)
    out = self.fc_1(out)
    out = self.act(out)
    out = self.fc(out)
    return out 

In [ ]:
def train_evaluate(parameterization):

    # Get neural net
    lstm = LSTM_Time_Series(parameterization).to(device)
    
    # train
    trained_net = net_train(net=lstm, train_loader=train_loader, parameters=parameterization, dtype=dtype, device=device)
    
    # return the accuracy of the model as it was trained in this run
    return evaluate(
        net=trained_net,
        data_loader=test_loader,
        dtype=dtype,
        device=device,
    )

In [ ]:
def hyperparam_optim():

  all_time = time.time()
  best_parameters, values, experiment, model = optimize(
      parameters=[
          {"name": "lr", "type": "range", "bounds": [1e-7, 1e-3], "log_scale": True},
          {"name": "num_epochs", "type": "range", "bounds": [10, 100]},
          {"name": "hidden_size", "type": "range", "bounds": [1, 500]},
          {"name": "optim_num", "type": "choice", "values": [0,1,2,3]},
          {"name": "act_num", "type": "choice", "values": [0,1,2,3,4]},
          {"name": "input_size", "type": "fixed", "value": 403},
          {"name": "num_layers", "type": "fixed", "value": 1},
          {"name": "num_classes", "type": "fixed", "value": 1},
      ],
      minimize = True,
      total_trials = 500, # total iteration for hyperparameter optimization
      evaluation_function=train_evaluate,
      objective_name='MSE',
  )
  print("total time : {}".format(time.time()-all_time))
  print("\n=======================\nbest_parameters\nlr : {}\nnum_epochs : {}\nhidden_size : {}\noptimization : {}\nactivation : {}\n".format(
      best_parameters["lr"],best_parameters["num_epochs"],best_parameters["hidden_size"],num2optim(best_parameters["optim_num"]),num2act(best_parameters["act_num"])))
  means, covariances = values
  print(means)
  print(covariances)

  best_objectives = np.array([[trial.objective_mean*100 for trial in experiment.trials.values()]])

  best_objective_plot = optimization_trace_single_method(
      y=np.minimum.accumulate(best_objectives, axis=1),
      title="Model performance vs. # of iterations",
      ylabel="MSE",
  )
  render(best_objective_plot)

  return (best_parameters, values, experiment, model)

In [ ]:
dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

garch = date2index(garch)
egarch = date2index(egarch)

In [ ]:
garch = garch.to_numpy()
egarch = egarch.to_numpy()
VIX = VIX.iloc[:, 1:2].to_numpy()

train_size = 3465

In [ ]:
# hyperparameter optimization for garch data
(X_train, y_train, X_test, y_test)= make_train_test(garch, VIX, train_size)

train_dataset = CustomDataset(X_train,y_train)
train_loader = DataLoader(train_dataset)
test_dataset = CustomDataset(X_test,y_test)
test_loader = DataLoader(test_dataset)

(best_parameters_g, values_g, experiment_g, model_g) = hyperparam_optim()

In [ ]:
# grach training
lr = best_parameters_g["lr"]
num_epochs = best_parameters_g["num_epochs"] * 100
optim_num = best_parameters_g["optim_num"]

(X_train_tensors_final_g,X_test_tensors_final_g,y_train_tensors_g,y_test_tensors_g) = make_train_test_tensor(garch, VIX, train_size)

lstm_g = LSTM_Time_Series(best_parameters_g).to(device)

loss_function_g = torch.nn.MSELoss()    # mean-squared error for regression
optimizer_g = int2optim_func(optim_num, lstm_g, lr)
start_time = time.time()

for epoch in range(num_epochs):
  outputs_g = lstm_g.forward(X_train_tensors_final_g.to(device)) #forward pass
  optimizer_g.zero_grad() #caluclate the gradient, manually setting to 0

  # obtain the loss function
  loss_g = loss_function_g(outputs_g, y_train_tensors_g.to(device))

  loss_g.backward() #calculates the loss of the loss function

  optimizer_g.step() #improve from loss, i.e backprop

print("total time : {}".format(time.time()-start_time))

# garch errors
(GT_g, DP_g) = visualize(garch, VIX, lstm_g, train_size)
(MSE_g, MAE_g, MAPE_g, RMSE_g) = errors(GT_g, DP_g, train_size)

In [ ]:
# hyperparameter optimization for egarch data
(X_train, y_train, X_test, y_test)= make_train_test(egarch, VIX, train_size)

train_dataset = CustomDataset(X_train,y_train)
train_loader = DataLoader(train_dataset)
test_dataset = CustomDataset(X_test,y_test)
test_loader = DataLoader(test_dataset)

(best_parameters_eg, values_eg, experiment_eg, model_eg) = hyperparam_optim()

In [ ]:
# egrach training
lr = best_parameters_eg["lr"]
num_epochs = best_parameters_eg["num_epochs"] * 100
optim_num = best_parameters_eg["optim_num"]

(X_train_tensors_final_eg,X_test_tensors_final_eg,y_train_tensors_eg,y_test_tensors_eg) = make_train_test_tensor(egarch, VIX, train_size)

lstm_eg = LSTM_Time_Series(best_parameters_eg).to(device)

loss_function_eg = torch.nn.MSELoss()    # mean-squared error for regression
optimizer_eg = int2optim_func(optim_num, lstm_eg, lr)
start_time = time.time()

for epoch in range(num_epochs):
  outputs_eg = lstm_eg.forward(X_train_tensors_final_eg.to(device)) #forward pass
  optimizer_eg.zero_grad() #caluclate the gradient, manually setting to 0

  # obtain the loss function
  loss_eg = loss_function_eg(outputs_eg, y_train_tensors_eg.to(device))

  loss_eg.backward() #calculates the loss of the loss function

  optimizer_eg.step() #improve from loss, i.e backprop

print("total time : {}".format(time.time()-start_time))

# egarch errors
(GT_eg, DP_eg) = visualize(garch, VIX, lstm_eg, train_size)
(MSE_eg, MAE_eg, MAPE_eg, RMSE_eg) = errors(GT_eg, DP_eg, train_size)